# Assignment 3 — Queries


In [1]:
from DbConnector import DbConnector
from statistics import median
import pandas as pd

conn = DbConnector()
db = conn.db
print("Connected to:", db.name)

def show_table(rows, limit=15):
    df = pd.DataFrame(rows)
    if len(df) > limit:
        display(df.head(limit))
        print(f"... {len(df) - limit} more rows not shown ...")
    else:
        display(df)
    return df


You are connected to the database: movies_db
-----------------------------------------------

Connected to: movies_db


## Q1 — Directors with 5 or more movies

This query looks at directors who have made at least five films and compares their box office performance and ratings.  
The idea is to identify who consistently directs successful movies both commercially and critically.  
It takes the crew data from `credits`, filters only those entries where the job is “Director”, and then joins them with the `movies` collection to get the revenue and average vote.  
Each director is grouped with all their movies, the number of films is counted, and the average TMDB rating is calculated.  
Finally, the median revenue is computed in Python since it is easier and avoids extra aggregation stages.


In [13]:
pipeline_q1 = [
    {"$unwind": "$crew"},
    {"$match": {"crew.job": "Director"}},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$group": {
        "_id": "$crew.name",
        "movie_count": {"$sum": 1},
        "revenues": {"$push": "$mv.revenue"},
        "avg_vote": {"$avg": "$mv.vote_average"}
    }},
    {"$match": {"movie_count": {"$gte": 5}}},
    {"$project": {
        "_id": 0,
        "director": "$_id",
        "movie_count": 1,
        "avg_vote": {"$round": ["$avg_vote", 2]},
        "revenues": 1
    }}
]

rows_q1 = list(db.credits.aggregate(pipeline_q1, allowDiskUse=True))
for r in rows_q1:
    valid = [x for x in (r.get("revenues") or []) if isinstance(x, (int, float))]
    r["median_revenue"] = float(median(valid)) if valid else None
    r.pop("revenues", None)

rows_q1_sorted = sorted(rows_q1, key=lambda x: (x["median_revenue"] or -1), reverse=True)[:20]
show_table(rows_q1_sorted)


,movie_count,director,avg_vote,median_revenue
0,5,Paul Morrissey,6.16,0.0
1,6,Carlo Verdone,6.25,0.0
2,5,Bruce Beresford,5.64,0.0
3,9,SABU,6.44,0.0
4,9,Abbas Kiarostami,5.80,0.0
5,5,Paul Schrader,5.26,0.0
6,5,Robert Greenwald,6.12,0.0
7,5,Neri Parenti,5.56,0.0
8,5,Peter Yates,6.24,0.0
9,8,Liu Chia-Liang,6.59,0.0


... 5 more rows not shown ...


,movie_count,director,avg_vote,median_revenue
0,5,Paul Morrissey,6.16,0.0
1,6,Carlo Verdone,6.25,0.0
2,5,Bruce Beresford,5.64,0.0
3,9,SABU,6.44,0.0
4,9,Abbas Kiarostami,5.80,0.0
5,5,Paul Schrader,5.26,0.0
6,5,Robert Greenwald,6.12,0.0
7,5,Neri Parenti,5.56,0.0
8,5,Peter Yates,6.24,0.0
9,8,Liu Chia-Liang,6.59,0.0


## Q2 — Actor pairs with at least three co-appearances

This query finds actor pairs that frequently appear together in movies.  
It unfolds the cast list twice for the same film to create all possible pairs and keeps only one order of each pair to avoid duplicates.  
The results are joined with `movies` to retrieve the rating of each film, and then the pairs are grouped by the two actor names.  
For every pair, the number of shared movies and their average rating are calculated, keeping only those who have worked together three or more times.


In [15]:
pipeline_q2 = [
    {"$project": {"movie_id": 1, "cast": 1}},
    {"$unwind": "$cast"},
    {"$project": {"movie_id": 1, "actor1": "$cast"}},
    {"$lookup": {
        "from": "credits",
        "localField": "movie_id",
        "foreignField": "movie_id",
        "as": "same"
    }},
    {"$unwind": "$same"},
    {"$unwind": "$same.cast"},
    {"$project": {
        "movie_id": 1,
        "a1_id": "$actor1.id",
        "a1_name": "$actor1.name",
        "a2_id": "$same.cast.id",
        "a2_name": "$same.cast.name"
    }},
    {"$match": {"$expr": {"$gt": ["$a2_id", "$a1_id"]}}},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$group": {
        "_id": {"a1": "$a1_name", "a2": "$a2_name"},
        "co_appearances": {"$sum": 1},
        "avg_vote": {"$avg": "$mv.vote_average"}
    }},
    {"$match": {"co_appearances": {"$gte": 3}}},
    {"$project": {
        "_id": 0,
        "actor_1": "$_id.a1",
        "actor_2": "$_id.a2",
        "co_appearances": 1,
        "avg_vote": {"$round": ["$avg_vote", 2]}
    }},
    {"$sort": {"co_appearances": -1, "avg_vote": -1}},
    {"$limit": 25}
]

rows_q2 = list(db.credits.aggregate(pipeline_q2, allowDiskUse=True))
show_table(rows_q2)


,co_appearances,actor_1,actor_2,avg_vote
0,18,Raymond Burr,Barbara Hale,4.97
1,16,Grey Griffin,Frank Welker,6.73
2,15,William R. Moses,Barbara Hale,5.39
3,14,Frank Welker,Mindy Cohn,6.69
4,13,Grey Griffin,Mindy Cohn,6.71
5,12,Walt Disney,Pinto Colvig,6.22
6,12,Raymond Burr,William R. Moses,4.88
7,11,Masako Nozawa,Hiromi Tsuru,6.83
8,11,Masako Nozawa,Naoko Watanabe,6.83
9,11,Mayumi Tanaka,Masako Nozawa,6.74


... 10 more rows not shown ...


,co_appearances,actor_1,actor_2,avg_vote
0,18,Raymond Burr,Barbara Hale,4.97
1,16,Grey Griffin,Frank Welker,6.73
2,15,William R. Moses,Barbara Hale,5.39
3,14,Frank Welker,Mindy Cohn,6.69
4,13,Grey Griffin,Mindy Cohn,6.71
5,12,Walt Disney,Pinto Colvig,6.22
6,12,Raymond Burr,William R. Moses,4.88
7,11,Masako Nozawa,Hiromi Tsuru,6.83
8,11,Masako Nozawa,Naoko Watanabe,6.83
9,11,Mayumi Tanaka,Masako Nozawa,6.74


## Q3 — Actors with the widest genre variety

The goal of this query is to highlight actors who have worked across the most diverse set of genres.  
After connecting each cast member with the corresponding movie information, the list of genres for each movie is expanded.  
The query then groups by actor name and collects a set of distinct genre names to count how many different ones appear.  
Only actors with at least ten movies are included, and the final list is sorted by the number of distinct genres.


In [4]:
pipeline_q3 = [
    {"$unwind": "$cast"},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$unwind": "$mv.genres"},
    {"$group": {
        "_id": "$cast.name",
        "movie_count": {"$sum": 1},
        "genres": {"$addToSet": "$mv.genres.name"}
    }},
    {"$match": {"movie_count": {"$gte": 10}}},
    {"$project": {
        "_id": 0,
        "actor": "$_id",
        "movie_count": 1,
        "genre_count": {"$size": "$genres"},
        "sample_genres": {"$slice": ["$genres", 5]}
    }},
    {"$sort": {"genre_count": -1, "movie_count": -1}},
    {"$limit": 25}
]

rows_q3 = list(db.credits.aggregate(pipeline_q3, allowDiskUse=True))
show_table(rows_q3)


,movie_count,actor,genre_count,sample_genres
0,72,Donald Sutherland,18,"[Fantasy, Adventure, Drama, Documentary, War]"
1,57,Alec Baldwin,18,"[Mystery, Romance, Drama, Documentary, War]"
2,52,Christopher Lloyd,18,"[Mystery, Action, Animation, Comedy, Crime]"
3,40,Billy Zane,18,"[Action, Animation, Crime, Science Fiction, Co..."
4,37,Michael Gambon,17,"[Mystery, Romance, War, Drama, Documentary]"
5,34,Michael McKean,17,"[Family, Adventure, Fantasy, War, Drama]"
6,62,John Goodman,16,"[Adventure, Fantasy, Family, Music, Thriller]"
7,57,Charlton Heston,16,"[War, Drama, Documentary, Adventure, Mystery]"
8,50,Robert Duvall,16,"[Action, Science Fiction, Comedy, Crime, TV Mo..."
9,43,John Hurt,16,"[Mystery, Romance, Drama, Adventure, Documentary]"


... 10 more rows not shown ...


,movie_count,actor,genre_count,sample_genres
0,72,Donald Sutherland,18,"[Fantasy, Adventure, Drama, Documentary, War]"
1,57,Alec Baldwin,18,"[Mystery, Romance, Drama, Documentary, War]"
2,52,Christopher Lloyd,18,"[Mystery, Action, Animation, Comedy, Crime]"
3,40,Billy Zane,18,"[Action, Animation, Crime, Science Fiction, Co..."
4,37,Michael Gambon,17,"[Mystery, Romance, War, Drama, Documentary]"
5,34,Michael McKean,17,"[Family, Adventure, Fantasy, War, Drama]"
6,62,John Goodman,16,"[Adventure, Fantasy, Family, Music, Thriller]"
7,57,Charlton Heston,16,"[War, Drama, Documentary, Adventure, Mystery]"
8,50,Robert Duvall,16,"[Action, Science Fiction, Comedy, Crime, TV Mo..."
9,43,John Hurt,16,"[Mystery, Romance, Drama, Adventure, Documentary]"


## Q4 — Top movie collections by total revenue

This query focuses on film collections or sagas and measures their financial success.  
It selects movies that belong to a collection, groups them by the collection name, and sums their total revenue.  
The aggregation also counts how many films belong to each collection and gathers all their ratings.  
The median of these ratings is calculated afterwards in Python, and the final result displays the ten most profitable collections.


In [5]:
pipeline_q4 = [
    {"$match": {"belongs_to_collection": {"$type": "object"}}},
    {"$group": {
        "_id": "$belongs_to_collection.name",
        "movie_count": {"$sum": 1},
        "total_revenue": {"$sum": "$revenue"},
        "ratings": {"$push": "$vote_average"},
        "first_date": {"$min": "$release_date"},
        "last_date": {"$max": "$release_date"}
    }},
    {"$match": {"movie_count": {"$gte": 3}}},
    {"$sort": {"total_revenue": -1}},
    {"$limit": 10}
]

rows_q4 = list(db.movies.aggregate(pipeline_q4, allowDiskUse=True))
for r in rows_q4:
    valid = [x for x in (r.get("ratings") or []) if isinstance(x, (int, float))]
    r["median_vote"] = float(median(valid)) if valid else None
    r.pop("ratings", None)

show_table(rows_q4)


,_id,movie_count,total_revenue,first_date,last_date,median_vote
0,Harry Potter Collection,8,7.707367e+09,2001-11-16,2011-07-07,7.50
1,Star Wars Collection,8,7.434495e+09,1977-05-25,2016-12-14,7.45
2,James Bond Collection,26,7.106970e+09,1962-10-04,2015-10-26,6.30
3,The Fast and the Furious Collection,8,5.125099e+09,2001-06-22,2017-04-12,6.65
4,Pirates of the Caribbean Collection,5,4.521577e+09,2003-07-09,2017-05-23,6.90
5,Transformers Collection,5,4.366101e+09,2007-06-27,2017-06-21,6.10
6,Despicable Me Collection,6,3.691070e+09,2010-07-08,2017-06-15,6.90
7,The Twilight Collection,5,3.342107e+09,2008-11-20,2012-11-13,5.80
8,Ice Age Collection,5,3.216709e+09,2002-03-10,2016-06-23,6.50
9,Jurassic Park Collection,4,3.031484e+09,1993-06-11,2015-06-09,6.35


,_id,movie_count,total_revenue,first_date,last_date,median_vote
0,Harry Potter Collection,8,7.707367e+09,2001-11-16,2011-07-07,7.50
1,Star Wars Collection,8,7.434495e+09,1977-05-25,2016-12-14,7.45
2,James Bond Collection,26,7.106970e+09,1962-10-04,2015-10-26,6.30
3,The Fast and the Furious Collection,8,5.125099e+09,2001-06-22,2017-04-12,6.65
4,Pirates of the Caribbean Collection,5,4.521577e+09,2003-07-09,2017-05-23,6.90
5,Transformers Collection,5,4.366101e+09,2007-06-27,2017-06-21,6.10
6,Despicable Me Collection,6,3.691070e+09,2010-07-08,2017-06-15,6.90
7,The Twilight Collection,5,3.342107e+09,2008-11-20,2012-11-13,5.80
8,Ice Age Collection,5,3.216709e+09,2002-03-10,2016-06-23,6.50
9,Jurassic Park Collection,4,3.031484e+09,1993-06-11,2015-06-09,6.35


## Q5 — Median runtime per decade and primary genre

This query explores how movie runtimes vary by decade and genre.  
The release year of each film is converted into a decade label such as “1980s” or “2000s”.  
The first genre listed in the movie data is treated as the primary genre for grouping purposes.  
Movies are grouped by both decade and genre, and for each group the total number of films is recorded.  
The median runtime is then computed in Python to give a clean summary of the central tendency for each category.


In [6]:
pipeline_q5 = [
    {"$addFields": {
        "year": {"$year": "$release_date"},
        "primary_genre": {
            "$cond": [
                {"$gt": [{"$size": {"$ifNull": ["$genres", []]}}, 0]},
                {"$arrayElemAt": ["$genres.name", 0]},
                None
            ]
        }
    }},
    {"$addFields": {
        "decade": {"$concat": [
            {"$toString": {"$multiply": [{"$floor": {"$divide": ["$year", 10]}}, 10]}},
            "s"
        ]}
    }},
    {"$group": {
        "_id": {"decade": "$decade", "genre": "$primary_genre"},
        "runtimes": {"$push": "$runtime"},
        "movie_count": {"$sum": 1}
    }}
]

rows_q5 = list(db.movies.aggregate(pipeline_q5, allowDiskUse=True))
for r in rows_q5:
    vals = [x for x in (r.get("runtimes") or []) if isinstance(x, (int, float))]
    r["median_runtime"] = float(median(vals)) if vals else None
    r.pop("runtimes", None)

rows_q5_sorted = sorted(rows_q5, key=lambda x: (x["_id"]["decade"], -(x["median_runtime"] or 0)))[:40]
show_table(rows_q5_sorted)


,_id,movie_count,median_runtime
0,"{'decade': '1870s', 'genre': 'Documentary'}",2,1.0
1,"{'decade': '1880s', 'genre': 'Documentary'}",4,1.0
2,"{'decade': '1890s', 'genre': 'Fantasy'}",8,1.5
3,"{'decade': '1890s', 'genre': 'Documentary'}",27,1.0
4,"{'decade': '1890s', 'genre': 'Action'}",2,1.0
5,"{'decade': '1890s', 'genre': None}",22,1.0
6,"{'decade': '1890s', 'genre': 'Comedy'}",9,1.0
7,"{'decade': '1890s', 'genre': 'History'}",1,1.0
8,"{'decade': '1890s', 'genre': 'Drama'}",2,1.0
9,"{'decade': '1890s', 'genre': 'Family'}",1,1.0


... 25 more rows not shown ...


,_id,movie_count,median_runtime
0,"{'decade': '1870s', 'genre': 'Documentary'}",2,1.0
1,"{'decade': '1880s', 'genre': 'Documentary'}",4,1.0
2,"{'decade': '1890s', 'genre': 'Fantasy'}",8,1.5
3,"{'decade': '1890s', 'genre': 'Documentary'}",27,1.0
4,"{'decade': '1890s', 'genre': 'Action'}",2,1.0
5,"{'decade': '1890s', 'genre': None}",22,1.0
6,"{'decade': '1890s', 'genre': 'Comedy'}",9,1.0
7,"{'decade': '1890s', 'genre': 'History'}",1,1.0
8,"{'decade': '1890s', 'genre': 'Drama'}",2,1.0
9,"{'decade': '1890s', 'genre': 'Family'}",1,1.0


## Q6 — Share of women in top-5 billed cast by decade

This query measures how the presence of women among the top-billed actors has changed over time.  
It keeps only cast entries where the order is less than five, which corresponds to the top of the cast list.  
For each movie, it counts how many of these actors are women using the `gender` value equal to 1.  
The ratio of female actors within the top five is then averaged by decade based on each movie’s release date.


In [7]:
pipeline_q6 = [
    {"$project": {"movie_id": 1, "cast": 1}},
    {"$unwind": "$cast"},
    {"$match": {"cast.order": {"$lt": 5}}},
    {"$group": {
        "_id": "$movie_id",
        "top5_total": {"$sum": 1},
        "top5_females": {"$sum": {"$cond": [{"$eq": ["$cast.gender", 1]}, 1, 0]}}
    }},
    {"$project": {
        "_id": 0,
        "movie_id": "$_id",
        "ratio": {
            "$cond": [{"$gt": ["$top5_total", 0]}, {"$divide": ["$top5_females", "$top5_total"]}, None]
        }
    }},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$addFields": {"year": {"$year": "$mv.release_date"}}},
    {"$addFields": {
        "decade": {"$concat": [
            {"$toString": {"$multiply": [{"$floor": {"$divide": ["$year", 10]}}, 10]}},
            "s"
        ]}
    }},
    {"$group": {"_id": "$decade", "avg_ratio": {"$avg": "$ratio"}, "movies": {"$sum": 1}}},
    {"$project": {"_id": 0, "decade": "$_id", "avg_ratio": {"$round": ["$avg_ratio", 3]}, "movies": 1}},
    {"$sort": {"decade": 1}}
]

rows_q6 = list(db.credits.aggregate(pipeline_q6, allowDiskUse=True))
show_table(rows_q6, limit=50)


,movies,decade,avg_ratio
0,11,1890s,0.000
1,18,1900s,0.000
2,8,1910s,0.450
3,20,1920s,0.214
4,122,1930s,0.298
5,205,1940s,0.306
6,327,1950s,0.281
7,343,1960s,0.257
8,466,1970s,0.220
9,583,1980s,0.240


,movies,decade,avg_ratio
0,11,1890s,0.000
1,18,1900s,0.000
2,8,1910s,0.450
3,20,1920s,0.214
4,122,1930s,0.298
5,205,1940s,0.306
6,327,1950s,0.281
7,343,1960s,0.257
8,466,1970s,0.220
9,583,1980s,0.240


## Q7 — Noir and neo-noir movies (regex search)

The goal of this query is to find movies described as noir or neo-noir in their overview or tagline.  
Since there is no text index in the database, regular expressions are used to match both words in a case-insensitive way.  
To avoid irrelevant titles, only movies with at least fifty votes are included.  
The output lists the top results sorted by rating.

In [8]:
pipeline_q7 = [
    {"$match": {
        "$or": [
            {"overview": {"$regex": "\\bneo-?noir\\b", "$options": "i"}},
            {"overview": {"$regex": "\\bnoir\\b", "$options": "i"}},
            {"tagline":  {"$regex": "\\bneo-?noir\\b", "$options": "i"}},
            {"tagline":  {"$regex": "\\bnoir\\b", "$options": "i"}}
        ],
        "vote_count": {"$gte": 50}
    }},
    {"$project": {"_id": 0, "title": 1, "release_date": 1, "vote_average": 1, "vote_count": 1}},
    {"$sort": {"vote_average": -1, "vote_count": -1}},
    {"$limit": 25}
]

rows_q7 = list(db.movies.aggregate(pipeline_q7, allowDiskUse=True))
show_table(rows_q7, limit=25)


,title,release_date,vote_average,vote_count
0,The Bad Sleep Well,1960-09-15,7.7,57
1,Drunken Angel,1948-04-27,7.7,54
2,Elevator to the Gallows,1958-01-29,7.6,85
3,Synchronicity,2015-07-22,5.7,114
4,Frank & Lola,2016-12-09,5.7,51


,title,release_date,vote_average,vote_count
0,The Bad Sleep Well,1960-09-15,7.7,57
1,Drunken Angel,1948-04-27,7.7,54
2,Elevator to the Gallows,1958-01-29,7.6,85
3,Synchronicity,2015-07-22,5.7,114
4,Frank & Lola,2016-12-09,5.7,51


## Q8 — Best director–actor collaborations

This query highlights pairs of directors and actors who have frequently worked together and achieved strong average ratings.  
It first extracts each movie’s director, joins the same movie’s cast to create all possible director–actor pairs,  
and then connects to the `movies` collection to include vote counts.  
Only movies with at least one hundred votes are considered to keep the sample meaningful,  
and only pairs with at least three collaborations are kept in the final ranking.


In [9]:
pipeline_q8 = [
    {"$unwind": "$crew"},
    {"$match": {"crew.job": "Director"}},
    {"$project": {"movie_id": 1, "director": "$crew.name"}},
    {"$lookup": {
        "from": "credits",
        "localField": "movie_id",
        "foreignField": "movie_id",
        "as": "cr"
    }},
    {"$unwind": "$cr"},
    {"$unwind": "$cr.cast"},
    {"$project": {"movie_id": 1, "director": 1, "actor": "$cr.cast.name"}},
    {"$lookup": {
        "from": "movies",
        "localField": "movie_id",
        "foreignField": "_id",
        "as": "mv"
    }},
    {"$unwind": "$mv"},
    {"$match": {"mv.vote_count": {"$gte": 100}}},
    {"$group": {
        "_id": {"director": "$director", "actor": "$actor"},
        "collabs": {"$sum": 1},
        "avg_vote": {"$avg": "$mv.vote_average"}
    }},
    {"$match": {"collabs": {"$gte": 3}}},
    {"$project": {
        "_id": 0,
        "director": "$_id.director",
        "actor": "$_id.actor",
        "collabs": 1,
        "avg_vote": {"$round": ["$avg_vote", 2]}
    }},
    {"$sort": {"avg_vote": -1, "collabs": -1}},
    {"$limit": 25}
]

rows_q8 = list(db.credits.aggregate(pipeline_q8, allowDiskUse=True))
show_table(rows_q8, limit=25)


""


""


## Q9 — Non-English languages connected to the United States

This query checks which non-English original languages appear most often in movies that have a connection to the United States.  
A connection is counted if the movie lists the United States either as a production country or as the origin of one of its production companies.  
The aggregation counts how many movies match each non-English language and returns the ten most common ones.


In [10]:
pipeline_q9 = [
    {"$match": {"original_language": {"$ne": "en"}}},
    {"$addFields": {
        "has_us_country": {"$gt": [{"$size": {
            "$filter": {
                "input": {"$ifNull": ["$production_countries", []]},
                "as": "c",
                "cond": {"$eq": ["$$c.iso_3166_1", "US"]}
            }}}, 0]},
        "has_us_company": {"$gt": [{"$size": {
            "$filter": {
                "input": {"$ifNull": ["$production_companies", []]},
                "as": "p",
                "cond": {"$eq": ["$$p.origin_country", "US"]}
            }}}, 0]}
    }},
    {"$match": {"$or": [{"has_us_country": True}, {"has_us_company": True}]}},
    {"$group": {"_id": "$original_language", "count": {"$sum": 1}}},
    {"$project": {"_id": 0, "language": "$_id", "count": 1}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

rows_q9 = list(db.movies.aggregate(pipeline_q9, allowDiskUse=True))
show_table(rows_q9, limit=10)


,count,language
0,112,fr
1,72,es
2,56,it
3,51,de
4,30,ja
5,15,pt
6,14,xx
7,12,nl
8,11,ru
9,10,zh


,count,language
0,112,fr
1,72,es
2,56,it
3,51,de
4,30,ja
5,15,pt
6,14,xx
7,12,nl
8,11,ru
9,10,zh


## Q10 — Per-user statistics: number of ratings, variance, and genre diversity

The last query calculates a few basic statistics for each user.  
It joins `ratings` with `links` and then `movies` to bring the genre information of every rated movie.  
Each user’s set of distinct genres is collected to count how varied their watched content is.  
The population standard deviation of the ratings is used to compute the variance.  
To keep the results stable, only users with at least twenty ratings are included.


In [ ]:
pipeline_q10 = [
    {"$lookup": {"from": "links", "localField": "movieId", "foreignField": "movieId", "as": "lk"}},
    {"$unwind": "$lk"},
    {"$lookup": {"from": "movies", "localField": "lk.tmdbId", "foreignField": "_id", "as": "mv"}},
    {"$unwind": "$mv"},
    {"$unwind": {"path": "$mv.genres", "preserveNullAndEmptyArrays": True}},
    {"$group": {
        "_id": "$userId",
        "ratings_count": {"$sum": 1},
        "genres": {"$addToSet": "$mv.genres.name"},
        "std_pop": {"$stdDevPop": "$rating"}
    }},
    {"$project": {
        "_id": 0,
        "userId": "$_id",
        "ratings_count": 1,
        "genre_count": {"$size": {"$setDifference": ["$genres", [None]]}},
        "variance": {
            "$cond": [{"$ne": ["$std_pop", None]}, {"$multiply": ["$std_pop", "$std_pop"]}, None]
        }
    }},
    {"$match": {"ratings_count": {"$gte": 20}}},
    {"$sort": {"genre_count": -1, "variance": -1, "ratings_count": -1}},
    {"$limit": 30}
]

rows_q10 = list(db.ratings.aggregate(pipeline_q10, allowDiskUse=True))
show_table(rows_q10, limit=30)
